# IMDB Ratings

Load packages

In [1]:
import pandas as pd
import requests
import gzip

# ignore warnings
import warnings
warnings.filterwarnings('ignore') 

# pandas defaults
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)

Get data

In [2]:
# retrieve zip files

url = ["https://datasets.imdbws.com/title.episode.tsv.gz", "https://datasets.imdbws.com/title.ratings.tsv.gz",
       'https://datasets.imdbws.com/title.basics.tsv.gz']
filename = []
for link in url:
    filename.append(link.split('/')[-1])
    
for name, link in zip(filename, url):
    with open(name, "wb") as f:
            r = requests.get(link)
            f.write(r.content)

In [3]:
# open files in dfs

dfs = []
for file in filename:
    with gzip.open(file) as f:
        df = pd.read_csv(f, sep='\t')
        dfs.append(df)

Split data in different dataframes and explore it

In [4]:
ep = dfs[0]
ep.head()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16


In [5]:
rat = dfs[1]
rat.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1656
1,tt0000002,6.1,201
2,tt0000003,6.5,1368
3,tt0000004,6.2,122
4,tt0000005,6.2,2150


In [6]:
name = dfs[2]
name.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


Prepare data

In [7]:
# join data in single dataframe
join = pd.merge(ep, rat, on='tconst')
join = pd.merge(join, name, on='tconst')
join = pd.merge(join, rat, left_on='parentTconst', right_on='tconst', suffixes=('_episode', '_show'))
join = pd.merge(join, name, left_on='parentTconst', right_on='tconst', suffixes=('_episode', '_show'))

# keep relevant fields
join = join[['tconst_episode', 'seasonNumber', 'episodeNumber', 'averageRating_episode', 'numVotes_episode',
             'titleType_episode', 'primaryTitle_episode', 'startYear_episode', 'runtimeMinutes_episode',
             'genres_episode', 'tconst_show', 'averageRating_show', 'numVotes_show', 'titleType_show',
             'primaryTitle_show', 'genres_show', 'startYear_show']]

# remove shows with less than 1000 votes
join_clean = join[join['numVotes_show']>1000]

Avoid duplicate show names

In [8]:
# get all shows
list_shows = join_clean.drop_duplicates(['primaryTitle_show','tconst_show'])\
                [['tconst_show','primaryTitle_show','startYear_show','numVotes_show']]

# shows with unique names
only_unique = list_shows.drop_duplicates(['primaryTitle_show'], keep=False)
only_unique['primaryTitle_show_new'] = only_unique['primaryTitle_show']

# shows with duplicated names
duplicated = list_shows[list_shows.duplicated(['primaryTitle_show'], keep=False)]
duplicated['primaryTitle_show_new'] = duplicated['primaryTitle_show']+' ('+duplicated['startYear_show'].astype(str)+')'

# join all data together
new_names = pd.concat([only_unique,duplicated]).sort_values('numVotes_show', ascending=False)\
                                .drop_duplicates('primaryTitle_show_new')[['tconst_show', 'primaryTitle_show_new']]

final = pd.merge(join_clean, new_names, on='tconst_show')

Save data

In [9]:
final.to_csv('imdb.csv')
final.head()

,tconst_episode,seasonNumber,episodeNumber,averageRating_episode,numVotes_episode,titleType_episode,primaryTitle_episode,startYear_episode,runtimeMinutes_episode,genres_episode,tconst_show,averageRating_show,numVotes_show,titleType_show,primaryTitle_show,genres_show,startYear_show,primaryTitle_show_new
0,tt0041951,1,9,7.4,54,tvEpisode,The Tenderfeet,1949,30,Western,tt0041038,7.8,2036,tvSeries,The Lone Ranger,Western,1949,The Lone Ranger
1,tt0635319,4,13,8.2,29,tvEpisode,A Broken Match,1954,23,Western,tt0041038,7.8,2036,tvSeries,The Lone Ranger,Western,1949,The Lone Ranger
2,tt0635320,5,21,8.8,27,tvEpisode,A Harp for Hannah,1957,23,Western,tt0041038,7.8,2036,tvSeries,The Lone Ranger,Western,1949,The Lone Ranger
3,tt0635321,1,41,7.7,30,tvEpisode,A Pardon for Curley,1950,23,Western,tt0041038,7.8,2036,tvSeries,The Lone Ranger,Western,1949,The Lone Ranger
4,tt0635322,3,27,7.3,22,tvEpisode,A Stage for Mademoiselle,1953,23,Western,tt0041038,7.8,2036,tvSeries,The Lone Ranger,Western,1949,The Lone Ranger
